![flaming dice](https://png.pngtree.com/element_our/sm/20180517/sm_5afd4577d28d1.jpg "PyCraps")


# PyCraps  ;)
### Nathan Rasch (nathan@obsidiantech.net)
#### © 2024

---

## Current features:
- Bankroll win/loss summary for each roll
- Win/loss messaging based on the results of each roll and the bets in play
- Pass line initial bet
- Pass line odds
- Field bets
- Place bet - all numbers shortcut
- Place bet - inner numbers shortcut
- Place bet - outer numbers shortcut
- Place bet - Iron Cross numbers shortcut
- Hardway bets for 4, 6, 8, and 10
- Roll counter
- Dice roll probability display for each roll
- End of game win/loss summary
- Unit tests
- C&E bets
- Smalls, talls, make 'em all bets


## To do
- Darkdside betting
- Lay bets (dark and light)
- Count for number of rolls since a seven
- Ability to change working status of bets
- Refactoring (I'm looking at you, payout_calculator_v2!)
- Remove debugging statements
- Graphical UI?

---


### Game code

In [600]:
# Imports
import random  # Importing the random module to generate random numbers and selections
import unittest  # Importing the unittest module to create and run unit tests
import IPython.display  # Importing the IPython.display module for displaying rich media in Jupyter notebooks
from sortedcontainers import SortedSet  # Importing SortedSet from sortedcontainers for maintaining a sorted collection of unique elements

In [602]:
# Globals
keep_playing = True  # Track if the player wishes to continue

In [563]:
class Tracker():
    """
    The Tracker class manages betting strategies, odds calculations, and financial tracking 

    Current examples:
    - Bet settings for the various craps bets such as the Iron Cross
    - Odds calculation in multiple formats (fractions, decimals, percents)
    - Financial tracking of winnings, losses, and bankroll
    - Methods to reset bets, set new bets, and display game stats
    - Method to "draw" the table showing bets placed with total amount at risk

    Attributes:
        point_numbers (list): List of numbers that can be bet on in the game.
        place_bet_payouts (dict): Payout ratios for place bets.
        pass_line_odds_payouts (dict): Payout ratios for pass line odds bets.
        dice_counts (dict): Count of possible outcomes for each sum of two dice.
        dice_odds_fractions (dict): Odds expressed as fractions.
        dice_odds_decimals (dict): Odds expressed as decimal numbers.
        dice_odds_percents (dict): Odds expressed as percentages.
        player_bankroll (int): Current amount of money the player has.
        total_winnings (int): Total amount won by the player.
        total_losses (int): Total amount lost by the player.
        table_bets (dict): Current bets placed on the table.
        roll_history (SortedSet): Ordered set to store the history of dice rolls.

    Methods:
        __init__(self): Initializes various class attributes including betting numbers, payout tables, and odds calculations. Sets up initial financial states and bet structures.
        show_odds_tables(self): Prints the odds tables in three formats: fractions, decimals, and percentages.
        get_odds_table(self, type): Returns the odds table based on the specified type ('fractions', 'decimals', or 'percents'). Raises a ValueError for invalid types.
        reset_table_bets(self, loss=True): Resets all bets to zero. If loss is True, it also updates the total losses and adjusts the player's bankroll accordingly.
        set_table_bets(self, bet_list, point=0): Sets or adjusts bets based on a string input representing multiple bets. Supports various betting strategies like 'inner', 'all', 'outer', etc. Adjusts bets to meet game-specific requirements (like divisibility by 3 for 6 and 8, or by 2 for odds on 5 and 9).
        print_table_bets(self): Returns a formatted string showing the current bets on the table, including the total sum of all bets.
        print_stats(self): Returns a formatted string displaying the player's total winnings, total losses, and current bankroll.
    """

    # Initialize the class
    def __init__(self):
        """
        Initializes the Tracker class for managing betting and game statistics in a dice game like Craps.
    
        - Sets up betting numbers for place bets.
        - Defines payout structures for place bets and pass line odds.
        - Calculates and stores various odds representations for dice outcomes:
          - Fractions, decimals, and percentages of each dice roll possibility.
        - Initializes player's financial tracking:
          - Bankroll, total winnings, and total losses.
        - Prepares a dictionary for current table bets, initially set via `reset_table_bets()`.
        - Uses SortedSet to keep a historical record of dice rolls in order.
    
        No parameters are required as all initial values are set within the method.
        """
        # Initialize a list of point numbers for betting
        self.point_numbers = ['4', '5', '6', '8', '9', '10']
        
        # Dictionary for place bet payouts, where keys are point numbers and values are payout ratios
        self.place_bet_payouts = {
            4: 9/5,  # Pays 9:5 for every dollar bet
            5: 7/5,  # Pays 7:5 for every dollar bet
            6: 7/6,  # Pays 7:6 for every dollar bet
            8: 7/6,  # Pays 7:6 for every dollar bet
            9: 7/5,  # Pays 7:5 for every dollar bet
            10: 9/5  # Pays 9:5 for every dollar bet
        }
    
        # Dictionary for pass line odds payouts, similar to place bet payouts but for different odds
        self.pass_line_odds_payouts = {
            4: 2/1,  # Pays 2:1 for every dollar bet
            5: 3/2,  # Pays 3:2 for every dollar bet
            6: 6/5,  # Pays 6:5 for every dollar bet
            8: 6/5,  # Pays 6:5 for every dollar bet
            9: 3/2,  # Pays 3:2 for every dollar bet
            10: 2/1  # Pays 2:1 for every dollar bet
        }
        
        # Dictionary where keys are the sum of two dice rolls, and values are the count of ways to roll that sum
        self.dice_counts = {
            2: 1,   # Only one way to roll a 2: (1,1)
            3: 2,   # Two ways to roll a 3: (1,2), (2,1)
            4: 3,   # Three ways to roll a 4: (1,3), (2,2), (3,1)
            5: 4,   # Four ways to roll a 5: (1,4), (2,3), (3,2), (4,1)
            6: 5,   # Five ways to roll a 6: (1,5), (2,4), (3,3), (4,2), (5,1)
            7: 6,   # Six ways to roll a 7: (1,6), (2,5), (3,4), (4,3), (5,2), (6,1)
            8: 5,   # Five ways to roll an 8: (2,6), (3,5), (4,4), (5,3), (6,2)
            9: 4,   # Four ways to roll a 9: (3,6), (4,5), (5,4), (6,3)
            10: 3,  # Three ways to roll a 10: (4,6), (5,5), (6,4)
            11: 2,  # Two ways to roll an 11: (5,6), (6,5)
            12: 1   # Only one way to roll a 12: (6,6)
        }
        
        # Calculate total possible dice roll combinations
        total_combinations = 6 * 6  
        
        # Calculate odds in different formats:
        # - Fractions: Represents odds as a fraction of total combinations
        # - Decimals: Rounded to 4 decimal places for readability
        # - Percents: Percentage chance rounded to 2 decimal places
        self.dice_odds_fractions = {k: f"{v}/{total_combinations}" for k, v in self.dice_counts.items()}
        self.dice_odds_decimals = {k: round(v / total_combinations, 4) for k, v in self.dice_counts.items()}
        self.dice_odds_percents = {k: f"{round(v / total_combinations * 100, 2)}%" for k, v in self.dice_counts.items()}

        # Initialize player's financial status
        self.player_bankroll = 0
        self.total_winnings = 0
        self.total_losses = 0

        # Initialize table bets dictionary
        self.table_bets = {}
        self.reset_table_bets()

        # Use SortedSet for maintaining roll history in order
        self.roll_history = SortedSet()
    
    # Method to display odds in all formats
    def show_odds_tables(self):
        print(self.dice_odds_fractions)
        print(self.dice_odds_decimals)
        print(self.dice_odds_percents)

    # Method to return a specific odds table based on the type argument
    def get_odds_table(self, type):
        if type == 'fractions':
            return self.dice_odds_fractions
        elif type == 'decimals':
            return self.dice_odds_decimals
        elif type == 'percents':
            return self.dice_odds_percents
        else:
            raise ValueError("Odds table must be fractions, decimals, or percents. Table type specified not found.")

    # Reset all bets to zero, optionally adjusting for loss and record the results if appropriate
    def reset_table_bets(self, loss=True):
        """
        Resets all bets to zero, optionally accounting for losses.
    
        - If 'loss' is True, it calculates and records the total loss from the current bets.
        - Updates the player's bankroll and total losses if applicable.
        - Prepares the bets dictionary for the next round by setting all bets to zero.
    
        Args:
            loss (bool): If True, the method will account for the current bets as losses. Default is True.
    
        Returns:
            int or None: Returns the total amount lost if 'loss' is True, otherwise returns None.
        """
        if loss:
            v = sum(self.table_bets.values())
            if not hasattr(self, 'total_losses'):
                print("total_losses attribute not found!")
            else:
                self.total_losses += v
                self.player_bankroll -= v

        # Reset the table bets for the next round
        self.table_bets = {}
        self.table_bets = {number: 0 for number in self.point_numbers}
        self.table_bets["Pass Line"] = 0

        # Return the amount lost, so we can display to the user
        if loss:
            return v
        
    # Set table bets based on a string formatted list of bets
    # Example bet_list:  8:5, 6:5, 10:10, field:5
    def set_table_bets(self, bet_list, point=0):
        """
        Sets or adjusts bets on the table according to the user's input.
    
        - Converts the point value to a string for consistency in bet handling.
        - Parses a comma-separated string of bets into individual bets.
        - Manages different types of bets like 'inner', 'all', 'outer', 'icross', 'odds', 'pass_line', and hard ways bets.
        - Ensures that bets on numbers 6 and 8 are divisible by 3 for proper payout calculations.
        - Adjusts odds bets on points 5 and 9 to be even numbers.
        - Handles exceptions to avoid crashes from invalid input or conversion issues.
    
        Args:
            bet_list (str): A string containing bets in the format 'type:amount', separated by commas.
            point (int): The current point if set, defaults to 0. Used for adjusting odds bets.
    
        Returns:
            None. Updates the `self.table_bets` dictionary in place.
        """
        point = str(point)  # Record the point if it has been set

        # Split the bets into individual components for processing
        split_1 = bet_list.split(",")
        split_1 = [x.strip() for x in split_1]

        # Process each bet
        for i in split_1:
            try:
                # Separate the type of bet from the amount of the bet
                num, bet = i.split(":")
                try:
                    # Define the numbers corresponding to the various table bets
                    common_numbers = [5, 6, 8, 9]
                    all_numbers = [4, 5, 6, 8, 9, 10]
                    outer_numbers = [4, 10]
                    field_numbers = [2, 3, 4, 9, 10, 11]
                    iron_cross_numbers = [5, 6, 8, 'field']

                    # Record the type of bet and the amount into the table dict
                    if num == "inner":
                        for x in common_numbers:
                            self.table_bets[str(x)] = int(bet)
                    elif num == "spread":
                        for x in all_numbers:
                            self.table_bets[str(x)] = int(bet)
                    elif num == "outer":
                        for x in outer_numbers:
                            self.table_bets[str(x)] = int(bet)
                    elif num == "icross":
                        for x in iron_cross_numbers:
                            self.table_bets[str(x)] = int(bet)
                    elif num == "pass_line":
                        self.table_bets['Pass Line'] = int(bet)
                    elif num in ['h4', 'h6', 'h8', 'h10']:
                        self.table_bets['Hard ' + num[1:]] = int(bet)
                    elif num in ['craps']:
                        self.table_bets['Any Craps'] = int(bet)
                    elif num in ['yo']:
                        self.table_bets['Yo Eleven'] = int(bet)
                    elif num in ['small', 'tall', 'all']:
                        # small, tall, and make 'em all bets can only happen before the point is made on the 1st roll of a new round
                        if point == "0":
                            self.table_bets[num] = int(bet)
                    # Assign any other bet not needing special logic to the table bet dictionary
                    # If the player puts in something crazy it won't register in the payout function and will be ignored
                    else:
                        self.table_bets[str(num)] = int(bet)
                    
                    # Ensure bets on 6 and 8 are divisible by 3 for correct payouts
                    for x in ['6', '8']:
                        if self.table_bets[x] > 0:
                            while self.table_bets[x] % 3 != 0:
                                self.table_bets[x] += 1

                    # Adjust pass line odds for points 5 and 9 to be even
                    if num == 'odds' and point in ['5', '9']:
                        while self.table_bets[num] % 2 != 0:
                            self.table_bets[num] += 1
                
                except ValueError:
                    pass  # Handle potential type conversion errors quietly
            except:
                pass  # General exception catch for any other issues

    # Method to format and print current table bets
    def print_table_bets(self):
        table = ""
        for k,v in self.table_bets.items():
            table += f"| {k.capitalize()} -> ${v} "
        table += f"| (Total: {sum(self.table_bets.values())})"
        return table

    # Method to print current statistics of winnings, losses, and bankroll
    def print_stats(self):
        return f"\t- Total winnings: ${self.total_winnings} | Total losses: ${self.total_losses} | Bankroll: ${self.player_bankroll}\n"

In [566]:
class Dealer():
    """
    The Dealer class simulates the operations of a casino dealer in a dice game like Craps. 
    It manages the game's flow, including rolling dice, setting and checking points, 
    handling bets, and calculating payouts. 

    Attributes:
        point_set (bool): Flag indicating if a point has been established.
        point_value (int): The number that has been set as the point if `point_set` is True.
        tracker (Tracker): An instance of Tracker to manage game stats and bets.
        odds (dict): Dictionary containing the odds of each dice roll outcome in fraction form.
        round_counter (int): Keeps track of the number of rolls in the current round.
        result_messages (list): Stores messages for game outcomes to be displayed.

    Methods:
        __init__(tracker): Initializes the Dealer with a Tracker instance.
        roll_dice(): Simulates rolling two dice and returns the result.
        new_round(): Starts a new round of the game, handling initial bets and bankroll setup.
        check_field_bet(roll): Checks if a field bet wins or loses based on the roll.
        payout_calculator_v2(bet, roll, d1, d2): Calculates payouts for various betting scenarios 
            based on the roll, including pass line, place, field, and hard way bets.
    """

    # Initialize the class
    def __init__(self, tracker):
        """
        Initializes the Dealer class for managing a dice game like Craps.
    
        - Sets up initial game state:
          - `point_set`: Flag for if a point has been established.
          - `point_value`: Stores the current point number.
        - Connects to the Tracker for game statistics management.
        - Prepares for displaying odds in fraction format.
        - Initializes a counter for tracking rolls within a round.
        - Sets up a list to collect outcome messages for each roll.
    
        Args:
            tracker (Tracker): An instance of Tracker to manage betting and statistics.
        """
        # Flag to indicate if a point has been established in the game
        self.point_set = False
        # The value set as the point when point_set becomes True
        self.point_value = 0
        # Reference to the Tracker object for managing game statistics and bets
        self.tracker = tracker
        # Get odds in fraction form from the Tracker for displaying to the player
        self.odds = self.tracker.get_odds_table('fractions')
        # Counter for the number of rolls in the current round
        self.round_counter = 0
        # List to store messages about the outcomes of each roll for display
        self.result_messages = []

    # Simulate rolling two six-sided dice and return the two dice plus the sum of the dice
    def roll_dice(self):
        """
        Simulates rolling two six-sided dice.
    
        - Generates two random integers between 1 and 6 to represent the dice.
        - Calculates the total of the two dice.
    
        Returns:
            tuple: A 3-tuple where the first two elements are the individual dice rolls,
                   and the third is their sum.
        """
        # Roll first die
        d1 = random.randint(1, 6)
        # Roll second die
        d2 = random.randint(1, 6)
        # Calculate the sum of the dice
        total = d1 + d2
    
        # Return a tuple containing the values of each die and their sum
        return (d1, d2, total)

    # Handles each new round of the game including initial setup for the 1st round
    def new_round(self):
        """
        Manages the flow of a new game round in a dice game like Craps.
        
        - Resets game state for a new round including point status and counters.
        - Handles initial player bankroll setup if it's zero.
        - Prompts for and sets the initial bet for the round.
        - Manages the gameplay loop for each roll within the round:
          - Displays the current round status.
          - Allows for bet adjustments before each roll.
          - Simulates a dice roll and updates game state based on the roll.
          - Calculates and displays payouts or losses.
          - Shows updated game statistics and roll history.
        
        Returns:
            None. This method runs the gameplay logic for one round.
        """
        # Reset game state for a new round
        self.point_set = False
        self.point_value = 0
        self.round_over = False
        self.round_counter = 0

        # Handle bankroll setup if it's zero
        if self.tracker.player_bankroll == 0:
            while True:
                br = input("Please enter your starting bankroll:")
                br = int(br)
                if br > 0:
                    self.tracker.player_bankroll = br
                    print(f"You are starting out with an initial bankroll of ${self.tracker.player_bankroll}.  Good luck!")
                    break
        
        # Set initial bet for the round
        while True:
            bet = input(f"How much do you wish to bet? [default $5]")
            if bet.isdigit():
                bet = int(bet)
            elif bet == "":
                bet = 5

            # Once the bet has been made record it
            if bet:
                print(f"You have bet ${bet}.  Good luck!")
                self.tracker.set_table_bets('pass_line:' + str(bet))
                break

        # Main game loop for the round
        while not self.round_over:
            # Track which round we are on for display
            self.round_counter += 1

            # Display round number and if this is the come out roll
            tmp = f"\n** Roll {self.round_counter}."
            if self.point_set:
                tmp += f"  Point is {self.point_value}"
            else:
                tmp += f"  Come out roll.  No point set.  WARNING:  Any bets made at this time are considered -working-."
            tmp += " **\n"
            print(tmp)

            # Display the current bets in play
            print("\tCurrent place bets:")
            print("\t" + self.tracker.print_table_bets())
            
            # Give the player a change to add/remove/modify their bets before the roll
            pbets = input(f"\n\tPlease enter any set bets you'd like to make or change: [enter for none]")
            if not pbets == "":
                # Record the player's new bets
                self.tracker.set_table_bets(pbets, self.point_value)
                # Display confirmation
                print("\n\tUpdated place bets:")
                print("\t" + self.tracker.print_table_bets() + "\n")

            # Roll the dice, record what was rolled
            input(f"Press any key to roll the dice! (Roll {self.round_counter})\n")
            d1, d2, total = self.roll_dice()
            self.tracker.roll_history.add(total)

            # Display roll message
            print(f"You rolled a {d1} and a {d2} for a total of {total}! (Odds are {self.odds[total]})")

            # Manage and display the results of the roll... Did the player win or lose, and if so how much?
            msgs = self.payout_calculator_v2(bet, total, d1, d2)
            for msg in msgs:
                print(msg)

            # Display the player win, loss, and bankroll info
            print(self.tracker.print_stats())
            # Display the roll history
            print(self.tracker.roll_history)

 
    def check_field_bet(self, roll):
        """
        Evaluates if the current roll results in a win or loss for a field bet.
    
        - Field bets win if the roll is 3, 4, 9, 10, or 11, with double payout for 2 or 12.
        - Updates the player's bankroll and total winnings/losses accordingly.
        - Appends a message about the field bet outcome to result_messages.
    
        Args:
            roll (int): The sum of the dice roll to evaluate.
    
        Returns:
            None (but appends to self.result_messages)
        """
        
        # Field bet winnings or losses calculation
        # Define numbers that win at 1:1 payout for a field bet
        field_numbers = [3, 4, 9, 10, 11]
        # Define numbers that win at 2:1 payout for a field bet
        double_numbers = [2, 12]
        # Initialize verb for message construction, assuming a win
        verb = "won"

        # Check if there's an active field bet
        if 'field' in self.tracker.table_bets.keys():
            # Retrieve the amount bet on the field
            bet = self.tracker.table_bets['field']
            if roll in field_numbers:
                # Player wins at 1:1 for these numbers
                self.tracker.total_winnings += bet
                self.tracker.player_bankroll += bet
                msg = f"\t- You have {verb} ${bet} on your field bet!"
            elif roll in double_numbers:
                # Player wins at 2:1 for these numbers
                self.tracker.total_winnings += bet * 2
                self.tracker.player_bankroll += bet * 2
                msg = f"\t- You have {verb} ${bet * 2} on your field bet!"
            else:
                # Player loses if the roll doesn't match field or double numbers
                verb = "lost"
                self.tracker.total_losses += bet
                self.tracker.player_bankroll -= bet
                msg = f"\t- You have {verb} ${bet} on your field bet!"

            # Add the outcome message to the list of result messages
            self.result_messages.append(msg)

    def payout_calculator_v2(self, bet, roll, d1, d2):
        """
        Calculates the wins/losses for the player based on the last dice roll.
        
        - Evaluates outcomes for pass line bets, place bets, field bets, and hard way bets.
        - Updates the player's bankroll, total winnings, and total losses.
        - Manages game state like setting points and ending rounds.
        
        Args:
            bet (int): The initial bet placed, though it uses 'Pass Line' bet from tracker.
            roll (int): The sum of the dice roll.
            d1 (int): The value of the first die.
            d2 (int): The value of the second die.
    
        Returns:
            list: A list of messages describing the outcome of the roll for each bet type.
        """
        # Comprehensive payout calculation for different bet types
        self.result_messages = []
        sroll = str(roll)
        # Use the pass line bet from the tracker, not the parameter 'bet' - Refactor To Do!
        bet = self.tracker.table_bets['Pass Line']
    
        # Handle come out roll scenarios
        if not self.point_set:
            if roll in (2, 3, 12):  # Craps numbers
                # Craps on come out roll
                self.result_messages.append(f"** Craps rolled on the come out; player loses ${bet}  :(\n")
                self.tracker.total_losses += bet
                self.tracker.player_bankroll -= bet
                self.round_over = True
                self.tracker.roll_history.add(roll)
            elif roll in (7, 11):  # Natural numbers
                # Natural on come out roll
                self.result_messages.append(f"** 7 or 11 rolled on the come out; player wins ${bet}!  :)\n")
                self.tracker.total_winnings += bet
                self.tracker.player_bankroll += bet
                self.round_over = True
                if roll == 7:  # Clear history if a 7 was rolled to start a new game cycle
                    self.tracker.roll_history.clear()
            else:
                # Point is set for numbers other than craps or natural
                self.point_set = True
                self.point_value = roll
                if self.tracker.table_bets[sroll] > 0:  # Return place bets if any were on the point
                    self.result_messages.append(f"\t Returning your place bet of ${self.tracker.table_bets[sroll]} for {roll}.")
                    self.tracker.table_bets[sroll] = 0
    
        # Handle point established scenarios
        elif self.point_set:
            if roll == 7:
                # Seven out, player loses all bets
                self.result_messages.append(f"** Played rolled a 7; player loses ${sum(self.tracker.table_bets.values())} total.  :(\n")
                self.tracker.reset_table_bets(loss=True)
                self.point_set = False
                self.round_over = True
                self.point_value = 0
                self.tracker.total_losses += sum(self.tracker.table_bets.values())
                self.tracker.player_bankroll -= sum(self.tracker.table_bets.values())
                self.tracker.roll_history.clear()
            else:
                # Track how many rolls since the last 7
                self.tracker.roll_history.add(roll)
            
            if roll == self.point_value:
                # Point hit, player wins
                self.result_messages.append(f"\n** Player has hit the point; player wins ${bet}!  :)\n")
                if 'odds' in self.tracker.table_bets.keys():  # Check if odds were placed on the pass line
                    tmp = int(self.tracker.pass_line_odds_payouts[roll] * self.tracker.table_bets['odds'])
                    self.result_messages.append(f"\t- You have won ${tmp} for your pass line odds bet of ${self.tracker.table_bets['odds']} on {roll}!")
                    bet += tmp  # Add odds winnings to the pass line win
                self.point_set = False
                self.round_over = True
                self.point_value = 0
                self.tracker.total_winnings += bet
                self.tracker.player_bankroll += bet
    
            # Check for place bets
            try:
                tmp = self.tracker.table_bets[str(roll)]
                if tmp:  # If there's a place bet, calculate and add winnings
                    tmp = int(self.tracker.table_bets[str(roll)] * self.tracker.place_bet_payouts[roll])
                    self.result_messages.append(f"\t- You have won ${tmp} for your place bet on {roll}!")
                    self.tracker.total_winnings += tmp
                    self.tracker.player_bankroll += tmp
            except:
                pass  # No place bet for this roll, or an error occurred
    
        # Handle always working bets like field bets
        self.check_field_bet(roll)
    
        # Handle hard ways bets
        key = str("Hard " + str(roll))
        if key in self.tracker.table_bets.keys():  # Check if there's a hard way bet for this number
            if d1 == d2:  # Hard way bets win only on doubles
                multi = 7 if d1 in [2, 5] else 9  # Payout multiplier for hard ways
                tmp = int(self.tracker.table_bets[key] * multi)
                self.result_messages.append(f"\t- You have won ${tmp} for your hard way bet on {roll}!")
                self.tracker.total_winnings += tmp
                self.tracker.player_bankroll += tmp
            else:
                tmp = int(self.tracker.table_bets[key])  # Player loses the bet but it stays active
                self.result_messages.append(f"\t- You have lost ${tmp} for your hard way bet on {roll}!  Keeping the bet up for the next roll.")
                self.tracker.total_losses += tmp
                self.tracker.player_bankroll -= tmp 

        # Handle any craps bets
        self.check_any_craps_bet(roll)

        # Handle any yo eleven bets
        self.check_eleven_bet(roll)

        # Handle any of the bonus bets
        self.check_bonus_bets()
        
        # Return the results of the bets based on the roll for display to the user
        return self.result_messages

    
    # Process any C&E bets (i.e. any craps or an eleven)
    def check_c_and_e_bet(self, roll):
        # C&E bet winnings or losses calculation
        # Define the craps numbers that win at 7:1 payout
        craps_numbers = [2, 3, 12]
        # Define the yo number that wins at 15:1 payout
        yo_numbers = [11]
        # Initialize verb for message construction, assuming a win
        verb = "won"

        # Check if there's an active field bet
        if 'ce' in self.tracker.table_bets.keys():
            # Retrieve the amount bet on the C&E
            bet = self.tracker.table_bets['ce']
            if roll in craps_numbers:
                # Player wins at 7:1 for these numbers
                self.tracker.total_winnings += bet * 7
                self.tracker.player_bankroll += bet * 7
                msg = f"\t- You have {verb} ${bet * 7} on your C&E bet!"
            elif roll in yo_numbers:
                # Player wins at 15:1 for a yo
                self.tracker.total_winnings += bet * 15
                self.tracker.player_bankroll += bet * 15
                msg = f"\t- You have {verb} ${bet * 15} on your field bet!"
            else:
                # Player loses if the roll doesn't match field or double numbers
                verb = "lost"
                self.tracker.total_losses += bet
                self.tracker.player_bankroll -= bet
                msg = f"\t- You have {verb} ${bet} on your C&E bet!"

            # Add the outcome message to the list of result messages
            self.result_messages.append(msg)
    
    # Process any craps bets (aka C)
    def check_any_craps_bet(self, roll):
        """
        Evaluates if the current roll results in a win or loss for an 'Any Craps' bet.
    
        - 'Any Craps' bets win if the roll is 2, 3, or 12, with a payout of 7:1.
        - Updates the player's bankroll and total winnings/losses accordingly.
        - Appends a message about the 'Any Craps' bet outcome to result_messages.
    
        Args:
            roll (int): The sum of the dice roll to evaluate.
    
        Returns:
            None (but appends to self.result_messages)
        """
        # Craps bet winnings or losses calculation
        # Define the craps numbers that win at 7:1 payout
        craps_numbers = [2, 3, 12]
        # Initialize verb for message construction, assuming a win
        verb = "won"

        # Check if there's an active craps bet
        if 'Any Craps' in self.tracker.table_bets.keys():
            # Retrieve the amount bet on the any craps
            bet = self.tracker.table_bets['Any Craps']
            if roll in craps_numbers:
                # Player wins at 7:1 for these numbers
                self.tracker.total_winnings += bet * 7
                self.tracker.player_bankroll += bet * 7
                msg = f"\t- You have {verb} ${bet * 7} on your any craps bet!"
            else:
                # Player loses if the roll doesn't match one of the craps numbers
                verb = "lost"
                self.tracker.total_losses += bet
                self.tracker.player_bankroll -= bet
                msg = f"\t- You have {verb} ${bet} on your any craps bet!"

            # Add the outcome message to the list of result messages
            self.result_messages.append(msg)
        
    # Process any 'yo' (i.e. 11) bets (aka E)
    def check_eleven_bet(self, roll):
        """
        Evaluates if the current roll results in a win or loss for a 'Yo Eleven' bet.
    
        - 'Yo Eleven' bets win only if the roll is 11, with a payout of 15:1.
        - Updates the player's bankroll and total winnings/losses accordingly.
        - Appends a message about the 'Yo Eleven' bet outcome to result_messages.
    
        Args:
            roll (int): The sum of the dice roll to evaluate.
    
        Returns:
            None (but appends to self.result_messages)
        """
        # Yo bet winnings or losses calculation
        # Define the yo number that wins at 15:1 payout
        yo_numbers = [11]
        # Initialize verb for message construction, assuming a win
        verb = "won"

        # Check if there's an active yo eleven bet
        if 'Yo Eleven' in self.tracker.table_bets.keys():
            # Retrieve the amount bet on the eleven
            bet = self.tracker.table_bets['Yo Eleven']
            if roll in yo_numbers:
                # Player wins at 15:1 for these numbers
                self.tracker.total_winnings += bet * 15
                self.tracker.player_bankroll += bet * 15
                msg = f"\t- You have {verb} ${bet * 15} on your Yo Eleven bet!"
            else:
                # Player loses if the roll doesn't match the yo number
                verb = "lost"
                self.tracker.total_losses += bet
                self.tracker.player_bankroll -= bet
                msg = f"\t- You have {verb} ${bet} on your Yo Eleven bet!"

            # Add the outcome message to the list of result messages
            self.result_messages.append(msg)


    # Process the of the bonus bets (i.e. small, tall, make 'em all)
    def check_bonus_bets(self):
        """
        Evaluates if the current roll results in a win or loss for bonus bets like 'Small', 'Tall', or 'All'.
    
        - Checks if all required numbers for 'Small', 'Tall', or 'All' have been rolled before a 7.
        - Updates the player's bankroll and total winnings/losses accordingly.
        - Appends a message about the bonus bet outcome to result_messages.
    
        Args:
            roll (int): The sum of the dice roll to evaluate.
    
        Returns:
            None (but appends to self.result_messages)
        """
        # Define the sets of numbers for bonus bets
        bonus_bets = {
            'small': [2, 3, 4, 5, 6],
            'tall': [8, 9, 10, 11, 12],
            'all': [2, 3, 4, 5, 6, 8, 9, 10, 11, 12]
        }
    
        # Payout odds for each bonus bet
        payouts = {
            'small': 30,
            'tall': 30,
            'all': 150
        }

        # Process each each bonus bet type
        for bet_type, numbers in bonus_bets.items():
            # See if the player has an active bet on the bonus bet type
            if bet_type in self.tracker.table_bets.keys():
                bet = self.tracker.table_bets[bet_type]
                # Check if all numbers for this bet type have been rolled
                if set(numbers).issubset(self.tracker.roll_history):
                    # Player wins if all numbers are rolled before a 7
                    winnings = bet * payouts[bet_type]
                    self.tracker.total_winnings += winnings
                    self.tracker.player_bankroll += winnings
                    # Remove the bet to prevent multiple wins
                    self.tracker.table_bets.pop(bet_type, None)
                    msg = f"\t- You have won ${winnings} on your {bet_type} bonus bet!"
                else:
                    # No win or loss message if not all numbers are rolled yet
                    msg = None
    
                # Append message if there's a win
                if msg:
                    self.result_messages.append(msg)



---
### Run the cell below to play the game   :)

In [570]:
# Run this to play the game   :)

# Instantiate the game objects
tracker = Tracker()
dealer = Dealer(tracker)

# Does the player wish to keep playing?
keep_playing = False

# While the user wants to continue keep playing rounds
while keep_playing:
    dealer.new_round()  

    pin = input("Do you wish to keep playing? [Y|n]")
    if pin.strip().lower() == "n":
        keep_playing = False

# Display end of game stats
print("End of game stats:\n")
print(f"\tTotal winnings: ${tracker.total_winnings}")
print(f"\tTotal losses: ${tracker.total_losses}")
print(f"\tFinal bankroll: ${tracker.player_bankroll}")

End of game stats:

	Total winnings: $0
	Total losses: $0
	Final bankroll: $0


---
### Unit Tests

In [574]:
# Tracker class unit tests
class TestTracker(unittest.TestCase):

    def setUp(self):
        self.tracker = Tracker()
        self.tracker.reset_table_bets()
    
    def test_init(self):
        # Check if all dictionaries are initialized correctly
        #print(len(self.tracker.point_numbers))
        #print(self.tracker.point_numbers)
        self.assertEqual(len(self.tracker.point_numbers), 6)
        self.assertEqual(len(self.tracker.dice_counts), 11)
        self.assertEqual(len(self.tracker.dice_odds_fractions), 11)
        self.assertEqual(len(self.tracker.dice_odds_decimals), 11)
        self.assertEqual(len(self.tracker.dice_odds_percents), 11)
        tmp = {number: 0 for number in self.tracker.point_numbers}
        tmp["Pass Line"] = 0
        self.assertEqual(self.tracker.table_bets, tmp)
        self.assertEqual(self.tracker.player_bankroll, 0)
        self.assertEqual(self.tracker.total_winnings, 0)
        self.assertEqual(self.tracker.total_losses, 0)

    def test_reset_table_bets(self):
        # Set some arbitrary bets
        self.tracker.table_bets = {'4': 10, '5': 20, '6': 30, '8': 30, '9': 20, '10': 10, 'Pass Line': 5}
        self.tracker.player_bankroll = 125
        self.tracker.total_losses = 0
        
        self.tracker.reset_table_bets()
        tmp = {number: 0 for number in self.tracker.point_numbers}
        tmp["Pass Line"] = 0
        self.assertEqual(self.tracker.table_bets, tmp)
        self.assertEqual(self.tracker.total_losses, 125)  # Total of all bets
        self.assertEqual(self.tracker.player_bankroll, 0)  # Bankroll reduced by total bets

        # Test reset without adding to losses
        self.tracker.table_bets = {'4': 10, '5': 20, '6': 30, '8': 30, '9': 20, '10': 10, 'Pass Line': 5}
        self.tracker.player_bankroll = 125
        self.tracker.total_losses = 125
        
        self.tracker.reset_table_bets(loss=False)
        tmp = {number: 0 for number in self.tracker.point_numbers}
        tmp["Pass Line"] = 0
        self.assertEqual(self.tracker.table_bets, tmp)
        self.assertEqual(self.tracker.total_losses, 125)  # Losses should not change
        self.assertEqual(self.tracker.player_bankroll, 125)  # Bankroll should not reduce by total bets

        # Test reset with adding to losses
        self.tracker.table_bets = {'4': 10, '5': 20, '6': 30, '8': 30, '9': 20, '10': 10, 'Pass Line': 5}
        self.tracker.player_bankroll = 125
        self.tracker.total_losses = 125

        self.tracker.reset_table_bets(loss=True)
        tmp = {number: 0 for number in self.tracker.point_numbers}
        tmp["Pass Line"] = 0
        self.assertEqual(self.tracker.table_bets, tmp)
        self.assertEqual(self.tracker.total_losses, 250)  # Losses should double
        self.assertEqual(self.tracker.player_bankroll, 0)  # Bankroll should reduce by total bets

    def test_set_table_bets_1(self):
        # Test setting bets with various inputs
        self.tracker.set_table_bets("8:5, 6:5")
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 6, '8': 6, '9': 0, '10': 0, 'Pass Line': 0})

    def test_set_table_bets_2(self):
        # Test 'inner' and 'all'
        self.tracker.set_table_bets("inner:5")
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 5, '6': 6, '8': 6, '9': 5, '10': 0, 'Pass Line': 0})

    def test_set_table_bets_3(self):
        self.tracker.set_table_bets("spread:5")
        self.assertEqual(self.tracker.table_bets, {'4': 5, '5': 5, '6': 6, '8': 6, '9': 5, '10': 5, 'Pass Line': 0})

    def test_set_table_bets_4(self):
        # Test setting bets with various inputs
        self.tracker.set_table_bets("8:5, 6:5, odds:5")
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 6, '8': 6, '9': 0, '10': 0, 'Pass Line': 0, 'odds': 5})

    def test_set_table_bets_5(self):
        # Test non-numeric input (should not affect existing bets)
        # We may use this for tracking world bets for example later on...
        self.tracker.set_table_bets("invalid:10")
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'invalid' : 10})
        self.tracker.set_table_bets("field:10")
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'invalid' : 10, 'field' : 10})

    def test_print_table_bets(self):
        self.tracker.table_bets = {'4': 10, '5': 20, '6': 30, '8': 30, '9': 20, '10': 10}
        expected_output = "| 4 -> $10 | 5 -> $20 | 6 -> $30 | 8 -> $30 | 9 -> $20 | 10 -> $10 | (Total: 120)"
        self.assertEqual(self.tracker.print_table_bets(), expected_output)

    def test_get_odds_table(self):
        # Test each type of odds table
        self.assertIsInstance(self.tracker.get_odds_table('fractions'), dict)
        self.assertIsInstance(self.tracker.get_odds_table('decimals'), dict)
        self.assertIsInstance(self.tracker.get_odds_table('percents'), dict)

        with self.assertRaises(ValueError):
            self.tracker.get_odds_table('invalid_type')

    def test_set_table_bet_pass_line(self):
        self.tracker.set_table_bets("pass_line:5")
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 5})
        

In [576]:
 # Dealer class unit tests
class TestDealer(unittest.TestCase):

    def setUp(self):
        self.tracker = Tracker()
        self.dealer = Dealer(self.tracker)
        self.tracker.player_bankroll = 100
        self.tracker.total_winnings = 0
        self.tracker.total_losses = 0
        self.dealer.point_set = False
        self.tracker.reset_table_bets(loss = False)
        self.bonus_bets = {
            'small': {'numbers': [2, 3, 4, 5, 6], 'payout': 30},
            'tall': {'numbers': [8, 9, 10, 11, 12], 'payout': 30},
            'all': {'numbers': [2, 3, 4, 5, 6, 8, 9, 10, 11, 12], 'payout': 150}
        }

    # Test bet is recorded correctly
    def test_field_bets_1(self):
        self.tracker.set_table_bets("field:10")
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'field' : 10})

    # Test bet is paid out correctly for 2 (i.e. double)
    def test_field_bets_2(self):
        self.tracker.set_table_bets("field:10")
        self.dealer.check_field_bet(2)
        self.assertEqual(self.dealer.result_messages, ['\t- You have won $20 on your field bet!'])
        self.assertEqual(self.tracker.player_bankroll, 120)
        self.assertEqual(self.tracker.total_winnings, 20)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test bet is paid out correctly
    def test_field_bets_3(self):
        self.tracker.set_table_bets("field:10")
        self.dealer.check_field_bet(10)
        self.assertEqual(self.dealer.result_messages, ['\t- You have won $10 on your field bet!'])
        self.assertEqual(self.tracker.player_bankroll, 110)
        self.assertEqual(self.tracker.total_winnings, 10)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test loss condition for non field number
    def test_field_bets_4(self):
        self.tracker.set_table_bets("field:10")
        self.dealer.check_field_bet(8)
        self.assertEqual(self.dealer.result_messages, ['\t- You have lost $10 on your field bet!'])
        self.assertEqual(self.tracker.player_bankroll, 90)
        self.assertEqual(self.tracker.total_winnings, 0)
        self.assertEqual(self.tracker.total_losses, 10)

    # Test loss condition for 7
    def test_field_bets_5(self):
        self.tracker.set_table_bets("field:10")
        self.dealer.check_field_bet(7)
        self.assertEqual(self.dealer.result_messages, ['\t- You have lost $10 on your field bet!'])
        self.assertEqual(self.tracker.player_bankroll, 90)
        self.assertEqual(self.tracker.total_winnings, 0)
        self.assertEqual(self.tracker.total_losses, 10)

    # Test simple win/loss conditions for come out bets
    def test_payout_calculator_v2_simple_win_loss(self):
        ####
        # 1st roll win - 7
        self.dealer.point_set = False
        self.tracker.set_table_bets('pass_line:10')
        #print(self.tracker.table_bets)
        #print(self.tracker.table_bets['pass_line'])
        msg = self.dealer.payout_calculator_v2(self.tracker.table_bets['Pass Line'], 7, d1=0, d2=0)
        self.assertEqual(msg, ['** 7 or 11 rolled on the come out; player wins $10!  :)\n'])

        ####
        # 1st roll loss - 3
        msg = self.dealer.payout_calculator_v2(self.tracker.table_bets['Pass Line'], 3, d1=0, d2=0)
        self.assertEqual(msg, ["** Craps rolled on the come out; player loses $10  :(\n"])
        self.dealer.point_set = False

        self.assertEqual(self.tracker.player_bankroll, 100)
        self.assertEqual(self.tracker.total_winnings, 10)
        self.assertEqual(self.tracker.total_losses, 10)

    # Test place bet is returned if point set == existing place bet + roll wins field bet
    def test_payout_calculator_v2_point_set_1(self):
        ####
        # 1st roll sets the point w/ field bet and place already on 4 from last round
        self.tracker.table_bets['4'] = 5
        self.tracker.table_bets['field'] = 5
        msg = self.dealer.payout_calculator_v2(5, 4, d1=0, d2=0)
        self.assertEqual(msg, ['\t Returning your place bet of $5 for 4.', '\t- You have won $5 on your field bet!'])
        self.assertEqual(self.tracker.player_bankroll, 105)
        self.assertEqual(self.tracker.total_winnings, 5)
        self.assertEqual(self.tracker.total_losses, 0)
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'field' : 5})

    # Test pass line win
    def test_payout_calculator_v2_point_set_2(self):
        ####
        # Point 8, 2nd roll 8
        self.tracker.set_table_bets('pass_line:5')
        self.dealer.point_set = True
        self.dealer.point_value = 8
        msgs = self.dealer.payout_calculator_v2(5, 8, d1=0, d2=0)
        self.assertEqual(msgs, ['\n** Player has hit the point; player wins $5!  :)\n'])
        self.assertEqual(self.tracker.player_bankroll, 105)
        self.assertEqual(self.tracker.total_winnings, 5)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test pass line loss
    def test_payout_calculator_v2_point_set_3(self):
        ####
        # Point 8, 2nd roll 7
        self.tracker.set_table_bets('pass_line:5')
        self.dealer.point_set = True
        self.dealer.point_value = 8
        msgs = self.dealer.payout_calculator_v2(5, 7, d1=0, d2=0)
        self.assertEqual(msgs, ['** Played rolled a 7; player loses $5 total.  :(\n'])
        self.assertEqual(self.tracker.player_bankroll, 95)
        self.assertEqual(self.tracker.total_winnings, 0)
        self.assertEqual(self.tracker.total_losses, 5)

    # Test pass line push
    def test_payout_calculator_v2_point_set_4(self):
        ####
        # Point 8, 2nd roll 6
        #print(f"self.tracker.player_bankroll = {self.tracker.player_bankroll}")
        self.dealer.point_set = True
        self.dealer.point_value = 8
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=0, d2=0)
        #print(f"self.tracker.player_bankroll = {self.tracker.player_bankroll}")
        self.assertEqual(msgs, [])
        self.assertEqual(self.tracker.player_bankroll, 100)
        self.assertEqual(self.tracker.total_winnings, 0)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test series of rolls that hit place bets and pass line bets
    def test_payout_calculator_v2_point_set_5(self):
        ####
        # Point 8, inner field bets, roll 6, roll 9, roll 8
        self.tracker.set_table_bets('pass_line:5')
        self.tracker.set_table_bets("inner:5")
        self.tracker.table_bets['8'] = 0
        self.dealer.point_set = True
        self.dealer.point_value = 8
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=0, d2=0)
        self.assertEqual(msgs, ['\t- You have won $7 for your place bet on 6!'])
        self.assertEqual(self.tracker.player_bankroll, 107)
        self.assertEqual(self.tracker.total_winnings, 7)
        self.assertEqual(self.tracker.total_losses, 0)

        msgs = self.dealer.payout_calculator_v2(5, 9, d1=0, d2=0)
        self.assertEqual(msgs, ['\t- You have won $7 for your place bet on 9!'])
        self.assertEqual(self.tracker.player_bankroll, 114)
        self.assertEqual(self.tracker.total_winnings, 14)
        self.assertEqual(self.tracker.total_losses, 0)

        msgs = self.dealer.payout_calculator_v2(5, 8, d1=0, d2=0)
        self.assertEqual(msgs, ['\n** Player has hit the point; player wins $5!  :)\n'])
        self.assertEqual(self.tracker.player_bankroll, 119)
        self.assertEqual(self.tracker.total_winnings, 19)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test series of rolls that hit place bets and then sevens out - Typical craps game...   ;)
    def test_payout_calculator_v2_point_set_6(self):
        ####
        # Point 8, inner field bets, roll 6, roll 9, roll 7
        # 5 at risk
        self.tracker.set_table_bets('pass_line:5')
        # 12 + 5 = 17 at risk
        self.tracker.set_table_bets("inner:5")
        # 17 - 6 = 11 at risk
        self.tracker.table_bets['8'] = 0
        
        self.dealer.point_set = True
        self.dealer.point_value = 8
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=0, d2=0)
        
        self.assertEqual(msgs, ['\t- You have won $7 for your place bet on 6!'])
        self.assertEqual(self.tracker.player_bankroll, 107)
        self.assertEqual(self.tracker.total_winnings, 7)
        self.assertEqual(self.tracker.total_losses, 0)

        msgs = self.dealer.payout_calculator_v2(5, 9, d1=0, d2=0)
        self.assertEqual(msgs, ['\t- You have won $7 for your place bet on 9!'])
        self.assertEqual(self.tracker.player_bankroll, 114)
        self.assertEqual(self.tracker.total_winnings, 14)
        self.assertEqual(self.tracker.total_losses, 0)

        #print("**********")
        #print(self.tracker.table_bets)
        #print(sum(self.tracker.table_bets.values()))
        #print("**********")
        msgs = self.dealer.payout_calculator_v2(5, 7, d1=0, d2=0)
        #21
        self.assertEqual(msgs, ['** Played rolled a 7; player loses $21 total.  :(\n'])
        self.assertEqual(self.tracker.player_bankroll, (114 - 21))
        self.assertEqual(self.tracker.total_winnings, 14)
        self.assertEqual(self.tracker.total_losses, 21)

    # Test Iron Cross field win
    def test_payout_calculator_v2_icross_1(self):
        ####
        # Iron cross
        self.tracker.set_table_bets("icross:5")
        self.assertEqual({'4': 0, '5': 5, '6': 6, '8': 6, '9': 0, '10': 0, 'field': 5}, self.tracker.table_bets)
        
        # win on field
        msgs = self.dealer.payout_calculator_v2(5, 4, d1=0, d2=0)
        self.assertEqual(msgs, ["\t- You have won $5 on your field bet!"])
        self.assertEqual(self.tracker.player_bankroll, 105)
        self.assertEqual(self.tracker.total_winnings, 5)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test Iron Cross place win and field loss
    def test_payout_calculator_v2_icross_1(self):
        # win on 6 - lose field
        self.tracker.set_table_bets("icross:5")
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=0, d2=0)
        #print(msgs)
        self.assertEqual(msgs, ['\t- You have won $7 for your place bet on 6!', '\t- You have lost $5 on your field bet!'])
        self.assertEqual(self.tracker.player_bankroll, 107)
        self.assertEqual(self.tracker.total_winnings, 12)
        self.assertEqual(self.tracker.total_losses, 5)

    # Test Iron Cross seven out
    def test_payout_calculator_v2_icross_1(self):
        self.tracker.set_table_bets('pass_line:5')
        self.dealer.point_set = True
        self.point_value = 10
        self.tracker.set_table_bets("icross:5")
        # lose on on 7 - lose field and place bets _ $22 at stake
        # print(self.tracker.table_bets)
        # print(sum(self.tracker.table_bets.values()))
        msgs = self.dealer.payout_calculator_v2(5, 7, d1=0, d2=0)
        # print(self.tracker.table_bets)
        print(self.dealer.result_messages)
        self.assertEqual(msgs, ['** Played rolled a 7; player loses $27 total.  :(\n'])
        self.assertEqual(self.tracker.player_bankroll, 100-22-5)
        self.assertEqual(self.tracker.total_winnings, 0)
        self.assertEqual(self.tracker.total_losses, 27)

    # Test pass line win with odds
    def test_pass_line_odds_1(self):
        self.tracker.set_table_bets('pass_line:5')
        self.dealer.point_set = True
        self.dealer.point_value = 10
        self.tracker.set_table_bets("odds:5")
        msgs = self.dealer.payout_calculator_v2(5, 10, d1=0, d2=0)
        #print(msgs)
        self.assertEqual(msgs, ['\n** Player has hit the point; player wins $5!  :)\n', '\t- You have won $10 for your pass line odds bet of $5 on 10!'])
        self.assertEqual(self.tracker.player_bankroll, 115)
        self.assertEqual(self.tracker.total_winnings, 15)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test pass line win with odds
    def test_pass_line_odds_2(self):
        self.tracker.set_table_bets('pass_line:5')
        self.dealer.point_set = True
        self.dealer.point_value = 5
        
        #print("\n\n")
        #print("________________test_pass_line_odds_2")
        self.tracker.set_table_bets("odds:5", 5)
        #print(self.tracker.table_bets)
        msgs = self.dealer.payout_calculator_v2(5, 5, d1=0, d2=0)
        #print(self.tracker.table_bets)
        #print(msgs)
        
        self.assertEqual(msgs, ['\n** Player has hit the point; player wins $5!  :)\n', '\t- You have won $9 for your pass line odds bet of $6 on 5!'])
        #['\n** Player has hit the point; player wins $5!  :)\n', '\t- You have won $9 for your pass line odds bet of $6 on 5!']
        self.assertEqual(self.tracker.player_bankroll, 114)
        self.assertEqual(self.tracker.total_winnings, 14)
        self.assertEqual(self.tracker.total_losses, 0)

    # Test pass line loss with odds
    def test_pass_line_odds_3(self):
        # 5 at risk
        self.tracker.set_table_bets('pass_line:5')
        self.dealer.point_set = True
        self.dealer.point_value = 5
        
        #print("\n\n")
        #print("________________test_pass_line_odds_2")
        # Will be bumped to 6 for payout ratio, 11 at risk
        #print(f"self.tracker.player_bankroll = {self.tracker.player_bankroll}")
        self.tracker.set_table_bets("odds:5", 5)
        #print(self.tracker.table_bets)
        msgs = self.dealer.payout_calculator_v2(5, 7, d1=0, d2=0)
        #print(self.tracker.table_bets)
        #print(msgs)
        
        self.assertEqual(msgs, ['** Played rolled a 7; player loses $11 total.  :(\n'])
        #['\n** Player has hit the point; player wins $5!  :)\n', '\t- You have won $9 for your pass line odds bet of $6 on 5!']
        self.assertEqual(self.tracker.player_bankroll, 100 - 11)
        self.assertEqual(self.tracker.total_winnings, 0)
        self.assertEqual(self.tracker.total_losses, 11)
    
    # Test hard way bets are recorded correctly
    def test_hard_ways_1(self):
        self.tracker.set_table_bets('pass_line:5')
        self.tracker.set_table_bets('h4:15')
        self.tracker.set_table_bets('h6:25')
        self.tracker.set_table_bets('h8:35')
        self.tracker.set_table_bets('h10:45')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 5, 'Hard 4' : 15, 'Hard 6' : 25, 'Hard 8' : 35, 'Hard 10' : 45})

    # Test hard way bet loss combo 1
    def test_hard_ways_1(self):
        self.tracker.set_table_bets('h4:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Hard 4' : 5})
        msgs = self.dealer.payout_calculator_v2(5, 4, d1=3, d2=1)
        self.assertEqual(msgs, ['\t- You have lost $5 for your hard way bet on 4!  Keeping the bet up for the next roll.'])

    # Test hard way bet loss combo 2
    def test_hard_ways_2(self):
        self.tracker.set_table_bets('h4:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Hard 4' : 5})
        msgs = self.dealer.payout_calculator_v2(5, 4, d1=1, d2=3)
        self.assertEqual(msgs, ['\t- You have lost $5 for your hard way bet on 4!  Keeping the bet up for the next roll.'])

    # Test hard way bet win
    def test_hard_ways_3(self):
        self.tracker.set_table_bets('h4:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Hard 4' : 5})
        msgs = self.dealer.payout_calculator_v2(5, 4, d1=2, d2=2)
        self.assertEqual(msgs, ['\t- You have won $35 for your hard way bet on 4!'])

    # Test hard way bet win
    def test_hard_ways_4(self):
        self.tracker.set_table_bets('h10:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Hard 10' : 5})
        msgs = self.dealer.payout_calculator_v2(5, 10, d1=2, d2=2)
        self.assertEqual(msgs, ['\t- You have won $35 for your hard way bet on 10!'])

    # Test hard way bet win
    def test_hard_ways_5(self):
        self.tracker.set_table_bets('h6:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Hard 6' : 5})
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=3, d2=3)
        self.assertEqual(msgs, ['\t- You have won $45 for your hard way bet on 6!'])

    # Test hard way bet win
    def test_hard_ways_6(self):
        self.tracker.set_table_bets('h8:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Hard 8' : 5})
        msgs = self.dealer.payout_calculator_v2(5, 8, d1=4, d2=4)
        self.assertEqual(msgs, ['\t- You have won $45 for your hard way bet on 8!'])

    # Initial roll is a not a hard way and no hard way bets set
    def test_hard_ways_7(self):
        self.tracker.set_table_bets('pass_line:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 5})
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=4, d2=2)
        self.assertEqual(msgs, [])

    # Initial roll is a a hard way and no hard way bets set
    def test_hard_ways_8(self):
        self.tracker.set_table_bets('pass_line:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 5})
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=3, d2=3)
        self.assertEqual(msgs, [])

    # Test any craps bet is recorded correctly
    def test_any_craps_1(self):
        self.tracker.set_table_bets('craps:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Any Craps': 5})

    # Test any craps win
    def test_any_craps_2(self):
        self.dealer.point_set = True
        self.tracker.set_table_bets('craps:1')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Any Craps': 1})
        msgs = self.dealer.payout_calculator_v2(5, 2, d1=1, d2=1)
        self.assertEqual(msgs, ['\t- You have won $7 on your any craps bet!'])

    # Test any craps loss
    def test_any_craps_3(self):
        self.dealer.point_set = True
        self.tracker.set_table_bets('craps:1')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Any Craps': 1})
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=3, d2=3)
        self.assertEqual(msgs, ['\t- You have lost $1 on your any craps bet!'])

    # Test yo bet is recorded correctly
    def test_yo_1(self):
        self.tracker.set_table_bets('yo:5')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Yo Eleven': 5})

    # Test yo win
    def test_yo_2(self):
        self.dealer.point_set = True
        self.tracker.set_table_bets('yo:1')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Yo Eleven': 1})
        msgs = self.dealer.payout_calculator_v2(5, 11, d1=5, d2=6)
        self.assertEqual(msgs, ['\t- You have won $15 on your Yo Eleven bet!'])

    # Test yo loss
    def test_yo_3(self):
        self.dealer.point_set = True
        self.tracker.set_table_bets('yo:1')
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, 'Yo Eleven': 1})
        msgs = self.dealer.payout_calculator_v2(5, 6, d1=3, d2=3)
        self.assertEqual(msgs, ['\t- You have lost $1 on your Yo Eleven bet!'])


    ###
    # Test if bonus bets are recorded correctly if made before the point is set
    def test_bonus_before_point(self):
        for bet_type in ['small', 'tall', 'all']:
            self.dealer.point_set = False
            self.tracker.reset_table_bets(loss=False)
            self.tracker.set_table_bets(f'{bet_type}:1')
            expected_bets = {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, bet_type: 1}
            self.assertEqual(self.tracker.table_bets, expected_bets)
            
    
    ###
    # Test if bonus bet is recorded correctly after point has already; been made
    # Bonus shouldn't bet set for these tests, because we are setting the point with the 6
    # Thus the bonus bet wasn't made before the 1st roll
    def test_bonus_after_point(self):
        # Test each type of bonus bet
        for type in ['small', 'tall', 'all']:
            # Clear out betting table
            self.tracker.reset_table_bets(loss=False)
            # Simulate point set at 8
            self.dealer.point_set = True
            self.dealer.point_value = 8
            # User tries to make a bonus after point is set
            self.tracker.set_table_bets(f"{type}:1", 6)
            # Bonus bet should not be recorded
            self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0})

    ###
    # Test bonus bet wins

    # Bonus bet helper
    def bonus_bet_helper(self, bet_type):
        # Initial setup
        self.dealer.point_set = False
        self.tracker.set_table_bets(f'{bet_type}:1')
        expected_bets = {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0, bet_type: 1}
        self.assertEqual(self.tracker.table_bets, expected_bets)

        # Set up point and roll history
        self.dealer.point_set = True
        self.dealer.point_value = 6
        for number in self.bonus_bets[bet_type]['numbers']:
            self.tracker.roll_history.add(number)

        # Check for win
        msgs = self.dealer.payout_calculator_v2(5, 4, d1=3, d2=1)
        expected_msg = f'\t- You have won ${self.bonus_bets[bet_type]["payout"]} on your {bet_type} bonus bet!'
        self.assertAlmostEqual(msgs, [expected_msg])

        # Check bet removal to avoid multiple wins
        self.assertEqual(self.tracker.table_bets, {'4': 0, '5': 0, '6': 0, '8': 0, '9': 0, '10': 0, 'Pass Line': 0})
        
        # Ensure no win message on subsequent call
        msgs = self.dealer.payout_calculator_v2(5, 4, d1=3, d2=1)
        self.assertEqual(msgs, [])

    # Test small bonus bet win
    def test_small_bonus_bet(self):
        self.bonus_bet_helper('small')

    # Test tall bonus bet win
    def test_tall_bonus_bet(self):
        self.bonus_bet_helper('tall')

    # Test all bonus bet win
    def test_all_bonus_bet(self):
        self.bonus_bet_helper('all')
        

In [579]:
# Add the test suites, and execute the unit tests
suite = unittest.TestSuite()
suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestTracker))
suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestDealer))

runner = unittest.TextTestRunner()
runner.run(suite)


.............................................
----------------------------------------------------------------------
Ran 45 tests in 0.072s

OK


['** Played rolled a 7; player loses $27 total.  :(\n']


<unittest.runner.TextTestResult run=45 errors=0 failures=0>